# import

In [ ]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


# directories


In [ ]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   

plot_dir = Path('.')/'plot'
plot_dir.mkdir(exist_ok=True,parents=True)

# read exemple

In [ ]:
import pandas as pd

In [ ]:
td001 =pd.read_csv(data_dir/'full-db'/'csv'/'td001_dpe.csv',dtype=str)


In [5]:
from td001_processing import postprocessing_td001
td001 = postprocessing_td001(td001)

In [6]:
td001.nom_methode_dpe_norm.value_counts()

3CL 2012                     5065922
FACTURE                      3858572
THBCE(RT2012)/THC(RT2005)     413012
DPE vierge                     58659
3CL 2005                       27488
NON DEFINI                      7767
Name: nom_methode_dpe_norm, dtype: int64

In [107]:
dept = '38'
td011 =pd.read_csv(data_dir/'full-db'/'csv'/'td011_installation_chauffage.csv',dtype=str)
td012 =pd.read_csv(data_dir/'full-db'/'csv'/'td012_generateur_chauffage.csv',dtype=str)
td013 =pd.read_csv(data_dir/'full-db'/'csv'/'td013_installation_ecs.csv',dtype=str)
td014 =pd.read_csv(data_dir/'full-db'/'csv'/'td014_generateur_ecs.csv',dtype=str)
td006 =pd.read_csv(data_dir/'full-db'/'csv'/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/'full-db'/'csv'/'td001_dpe.csv',dtype=str)
td005 =pd.read_csv(data_dir/'full-db'/'csv'/'td005_fiche_technique.csv',dtype=str)

In [108]:
import pandas as pd
import numpy as np
from assets_orm import DPEMetaData
from td001_merge import merge_td001_dpe_id_system

td013_types = {'td013_installation_ecs_id': 'str',
 'td006_batiment_id': 'str',
 'tr005_type_installation_ecs_id': 'str',
 'nombre_appartements_echantillon': 'float',
 'surface_habitable_echantillon': 'float',
 'becs': 'float',
 'tv039_formule_becs_id': 'category',
 'surface_alimentee': 'float',
 'td001_dpe_id': 'str'}


td014_types = {'td014_generateur_ecs_id': 'str',
 'td013_installation_ecs_id': 'str',
 'tr004_type_energie_id': 'str',
 'tv045_conversion_kwh_co2_id': 'category',
 'tv046_evaluation_contenu_co2_reseaux_id': 'category',
 'coefficient_performance': 'float',
 'tv032_coefficient_performance_id': 'category',
 'rpn': 'float',
 'qp0': 'float',
 'puissance_veilleuse': 'float',
 'tv036_puissance_veilleuse_id': 'category',
 'tv037_puissance_necessaire_production_ecs_id': 'category',
 'puissance_nominale': 'float',
 'tv038_puissance_nominale_id': 'category',
 'tv040_rendement_distribution_ecs_id': 'category',
 'tv047_rendement_generation_ecs_id': 'category',
 'volume_stockage': 'float',
 'tv048_rendement_stockage_ecs_id': 'category',
 'tv049_perte_stockage_ecs_id': 'category',
 'tv041_coefficient_emplacement_fonctionnement_id': 'category',
 'tv043_pertes_stockage_id': 'category',
 'tv019_fecs_id': 'category',
 'tv027_pertes_recuperees_ecs_id': 'category',
 'td001_dpe_id': 'str'}

def merge_td013_tr_tv(td013):
    meta = DPEMetaData()
    table = td013.copy()
    table = meta.merge_all_tr_table(table)
    table = meta.merge_all_tv_table(table)
    table = table.astype(td013_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table

def merge_td014_tr_tv(td014):
    meta = DPEMetaData()
    table = td014.copy()
    table = meta.merge_all_tr_table(table)
    table = meta.merge_all_tv_table(table)
    table = table.astype(td014_types)
    table = table.rename(columns={'id': 'td013_installation_ecs_id'})

    return table

td001,td006,td011, td012,td013,td014 = merge_td001_dpe_id_system(td001, td006, td011, td012,td013,td014)

td013 = merge_td013_tr_tv(td013)

td014 = merge_td014_tr_tv(td014)

In [109]:
from utils import strip_accents,affect_lib_by_matching_score

In [7]:
td013

NameError: name 'td013' is not defined

In [110]:
table = td014.copy()

table = table.merge(td013[['tr005_description','td013_installation_ecs_id','surface_habitable_echantillon']],on = 'td013_installation_ecs_id')

is_chaudiere = table.rpn>0


gen_ecs_concat_txt_desc = table["tv027_Type d'installation"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc.loc[is_chaudiere] += 'chaudiere '

gen_ecs_concat_txt_desc += table['tv027_Type de système'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv032_Type de Générateur"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv036_Type de génération'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv037_Type de Production'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv040_Type de générateur'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv040_Type d'installation"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tr004_description"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table["tv045_Energie"].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tv047_Type de Générateur'].astype('string').replace(np.nan, '') + ' '
gen_ecs_concat_txt_desc += table['tr005_description'].astype('string').replace(np.nan, '') + ' '

gen_ecs_normalized_lib_matching_dict = {
    "ECS thermodynamique electrique(PAC ou ballon)": [('pompe a chaleur','pac','thermodynamique', 'air extrait','air exterieur'),
                                          ('electricite', 'electrique')],
    "ballon a accumulation electrique": [('ballon','classique','accumulation'),('electricite', 'electrique')],
    "ecs electrique indeterminee": [('electricite', 'electrique')],
    "ecs instantanee electrique": ['instantanee', ('electricite', 'electrique')],

    'chaudiere mixte gaz': ["chaudiere", 'mixte', "gaz"],
    'chaudiere mixte fioul': ["chaudiere", "mixte", "fioul"],
    'chaudiere mixte bois': [("bois", "biomasse")],

    'chauffe-eau gaz independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl"],

    'chauffe-eau fioul independant': [("individuelle ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "fioul"],
    "ecs collective reseau chaleur": ["reseau", "chaleur"],

    'chaudiere gaz': ["chaudiere", "gaz"],
    'chaudiere gpl': ["chaudiere", "gpl"],

    'chaudiere fioul': ["chaudiere", "fioul"],

}
solaire_dict=dict()
for k,v in gen_ecs_normalized_lib_matching_dict.items():
    k_solaire = 'ecs solaire thermique et '+k
    solaire_dict[k_solaire]=v +['avec solaire']
gen_ecs_normalized_lib_matching_dict.update(solaire_dict)
gen_ecs_lib_simp_dict={'ecs electrique indeterminee': 'ecs electrique indeterminee',
 'chaudiere gaz': 'chaudiere gaz',
 'ballon a accumulation electrique': 'ecs à effet joule electrique',
 'chaudiere mixte gaz': 'chaudiere gaz',
 'ECS thermodynamique electrique(PAC ou ballon)':  'ECS thermodynamique electrique(PAC ou ballon)',
 'non affecte':'non affecte',
 'chaudiere fioul': 'chaudiere fioul',
 'chaudiere mixte fioul':'chaudiere fioul',
 'chaudiere mixte bois': 'chaudiere mixte bois',
 'ecs collective reseau chaleur':'ecs collective reseau chaleur',
 'ecs instantanee electrique': 'ecs à effet joule electrique',
 'chauffe-eau gaz independant':  'chauffe-eau gaz independant',
 'chauffe-eau fioul independant':  'chauffe-eau fioul independant',
                      }
solaire_dict=dict()
for k,v in gen_ecs_lib_simp_dict.items():
    k_solaire = 'ecs thermique solaire et '+k
    v_solaire = 'ecs thermique solaire et '+v
    solaire_dict[k_solaire]=v_solaire
gen_ecs_lib_simp_dict.update(solaire_dict)

sys_principal_scores = {'thermodynamique':5,
                       'solaire':4,
                       'chaudiere':3,
                       'ballon a accumulation':2,
                       'electrique indeterminee':1,
                       'indépendant':0}

sys_principal_score_lib=dict()
for k in list(gen_ecs_normalized_lib_matching_dict.keys()):
    sys_principal_score_lib[k]=0
    for term,score in sys_principal_scores.items():
        if term in k :
            sys_principal_score_lib[k]+=score

from utils import clean_str,strip_accents,affect_lib_by_matching_score

gen_ecs_concat_txt_desc = gen_ecs_concat_txt_desc.str.lower().apply(lambda x: strip_accents(x))

table['gen_ecs_concat_txt_desc'] = gen_ecs_concat_txt_desc

table['gen_ecs_concat_txt_desc'] = table['gen_ecs_concat_txt_desc'].apply(lambda x: clean_str(x))

# calcul gen_ecs_lib_infer par matching score text.
unique_gen_ecs = table.gen_ecs_concat_txt_desc.unique()
gen_ecs_lib_infer_dict = {k: affect_lib_by_matching_score(k, gen_ecs_normalized_lib_matching_dict) for k in
                         unique_gen_ecs}
table['gen_ecs_lib_infer'] = table.gen_ecs_concat_txt_desc.replace(gen_ecs_lib_infer_dict)
is_pac = table.coefficient_performance>2
table.loc[is_pac,'gen_ecs_lib_infer']="ECS thermodynamique electrique(PAC ou ballon)"

table['gen_ecs_lib_infer_simp'] = table.gen_ecs_lib_infer.replace(gen_ecs_lib_simp_dict)

table['type_energie_ecs']=table['tr004_description']

In [123]:
sys_ecs_principal_rename = {
    'td001_dpe_id': 'td001_dpe_id',
    'gen_ecs_lib_infer': 'sys_ecs_principal_gen_ecs_lib_infer',
    'gen_ecs_lib_infer_simp': 'sys_ecs_principal_gen_ecs_lib_infer_simp',
    'type_energie_ecs': 'sys_ecs_principal_type_energie_ecs',
    'consommation_ecs': 'sys_ecs_principal_consommation_ecs',
    "tv025_Type d'installation": 'sys_ecs_principal_type_installation_ecs',
    'nombre_generateurs': 'sys_ecs_principal_nb_generateur'
}

sys_ecs_secondaire_rename = {
    'td001_dpe_id': 'td001_dpe_id',
    'gen_ecs_lib_infer': 'sys_ecs_secondaire_gen_ecs_lib_infer',
    'gen_ecs_lib_infer_simp': 'sys_ecs_secondaire_gen_ecs_lib_infer_simp',
    'type_energie_ecs': 'sys_ecs_secondaire_type_energie_ecs',
    'consommation_ecs': 'sys_ecs_secondaire_consommation_ecs',
    "tv025_Type d'installation": 'sys_ecs_secondaire_type_installation_ecs',
    'nombre_generateurs': 'sys_ecs_secondaire_nb_generateur'
}

sys_ecs_tertiaire_concat_rename = {
    'td001_dpe_id': 'td001_dpe_id',
    'gen_ecs_lib_infer': 'sys_ecs_tertiaire_gen_ecs_lib_infer_concat',
    'gen_ecs_lib_infer_simp': 'sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat',
    'type_energie_ecs': 'sys_ecs_tertiaire_type_energie_ecs_concat',
    'consommation_ecs': 'sys_ecs_tertiaire_consommation_ecs',
    "tv025_Type d'installation": 'sys_ecs_tertiaire_type_installation_ecs_concat',
    'nombre_generateurs': 'sys_ecs_tertiaire_nb_generateurs'
}




table['nombre_generateurs'] = 1

cols = ['td001_dpe_id', 'gen_ecs_lib_infer_simp', 'gen_ecs_lib_infer', 'type_energie_ecs']
cols += ['tr005_description','surface_habitable_echantillon', 'nombre_generateurs', 'id_unique']

agg_cols = ['td001_dpe_id', 'gen_ecs_lib_infer', 'tr005_description']

table['id_unique']=table.td001_dpe_id + table.gen_ecs_lib_infer.astype('string') +table.tr005_description.astype('string')

is_unique = table.groupby('td001_dpe_id').td001_dpe_id.count() == 1
ist_unique = is_unique[is_unique].index

table_gen_unique = table.loc[table.td001_dpe_id.isin(ist_unique)][cols]

table_gen_multiple = table.loc[~table.td001_dpe_id.isin(ist_unique)][cols]

table_gen_multiple = table.loc[~table.td001_dpe_id.isin(ist_unique)][cols]

table_gen_multiple[agg_cols] = table_gen_multiple[agg_cols].astype(str)

agg = table_gen_multiple.groupby(agg_cols).agg({
    'gen_ecs_lib_infer_simp': 'first',
    'type_energie_ecs': 'first',
    'surface_habitable_echantillon': 'sum',
    "nombre_generateurs": 'sum'

}).reset_index()

agg['id_unique'] = agg.td001_dpe_id + agg.gen_ecs_lib_infer + agg.tr005_description



sys_principal = agg.sort_values('consommation_ecs', ascending=False).drop_duplicates(subset='td001_dpe_id')

id_sys_principal = sys_principal.id_unique.unique().tolist()

sys_secondaires = agg.loc[~agg.id_unique.isin(id_sys_principal)]

sys_secondaire = sys_secondaires.sort_values('consommation_ecs', ascending=False).drop_duplicates(
    'td001_dpe_id')

id_sys_secondaire = sys_secondaire.id_unique.unique().tolist()

sys_tertiaires = agg.loc[~agg.id_unique.isin(id_sys_principal + id_sys_secondaire)]

sys_tertiaire_concat = sys_tertiaires.groupby('td001_dpe_id').agg({
    'gen_ecs_lib_infer': lambda x: ' + '.join(list(set(x))),

    'gen_ecs_lib_infer_simp': lambda x: ' + '.join(list(set(x))),
    'type_energie_ecs': lambda x: ' + '.join(list(set(x))),
    'consommation_ecs': 'sum',
    'besoin_ecs_infer': 'sum',
    "tv025_Type d'installation": lambda x: ' + '.join(list(set(x))),
    'nombre_generateurs': 'sum'
}).reset_index()

sys_principal = sys_principal.append(table_gen_unique[sys_principal.columns])

sys_principal = sys_principal.rename(columns=sys_ecs_principal_rename)[sys_ecs_principal_rename.values()]

sys_secondaire = sys_secondaire.rename(columns=sys_ecs_secondaire_rename)[sys_ecs_secondaire_rename.values()]

sys_tertiaire_concat = sys_tertiaire_concat.rename(columns=sys_ecs_tertiaire_concat_rename)[
    sys_ecs_tertiaire_concat_rename.values()]

td001_sys_ecs = td001[['td001_dpe_id']].merge(sys_principal, on='td001_dpe_id', how='left')
td001_sys_ecs = td001_sys_ecs.merge(sys_secondaire, on='td001_dpe_id', how='left')
td001_sys_ecs = td001_sys_ecs.merge(sys_tertiaire_concat, on='td001_dpe_id', how='left')
nb_installation = td013.groupby('td001_dpe_id').td013_installation_ecs_id.count().to_frame(
    'nombre_installations')
td001_sys_ecs = td001_sys_ecs.merge(nb_installation, on='td001_dpe_id', how='left')

cols_end = sys_principal.columns.tolist() + sys_secondaire.columns.tolist() + sys_tertiaire_concat.columns.tolist()
cols_end = np.unique(cols_end).tolist()

td001_sys_ecs['nombre_generateur_total'] = td001_sys_ecs.sys_ecs_principal_nb_generateur
td001_sys_ecs['nombre_generateur_total'] += td001_sys_ecs.sys_ecs_secondaire_nb_generateur.fillna(0)
td001_sys_ecs['nombre_generateur_total'] += td001_sys_ecs.sys_ecs_tertiaire_nb_generateurs.fillna(0)


cols = ['sys_ecs_principal_type_energie_ecs',
        'sys_ecs_secondaire_type_energie_ecs',
        'sys_ecs_tertiaire_type_energie_ecs_concat']

td001_sys_ecs['mix_energetique_ecs'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                               is_unique=True, is_sorted=True)

cols = ['sys_ecs_principal_type_installation_ecs',
        'sys_ecs_secondaire_type_installation_ecs',
        'sys_ecs_tertiaire_type_installation_ecs_concat']

td001_sys_ecs['type_installation_ecs_concat'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                                        is_unique=True, is_sorted=True)

cols = ['sys_ecs_principal_gen_ecs_lib_infer',
        'sys_ecs_secondaire_gen_ecs_lib_infer',
        'sys_ecs_tertiaire_gen_ecs_lib_infer_concat']

td001_sys_ecs['gen_ecs_lib_infer_concat'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                             is_unique=True, is_sorted=True)

cols = ['sys_ecs_principal_gen_ecs_lib_infer_simp',
        'sys_ecs_secondaire_gen_ecs_lib_infer_simp',
        'sys_ecs_tertiaire_gen_ecs_lib_infer_simp_concat']

td001_sys_ecs['gen_ecs_lib_infer_simp_concat'] = concat_string_cols(td001_sys_ecs, cols=cols, join_string=' + ',
                                                                  is_unique=True, is_sorted=True)

isnull = td001_sys_ecs.sys_ecs_principal_nb_generateur.isnull()
is_multiple_install = td001_sys_ecs.nombre_installations > 1
td001_sys_ecs.loc[isnull, 'configuration_sys_ecs'] = pd.NA
td001_sys_ecs.loc[~isnull, 'configuration_sys_ecs'] = 'générateur unique/installation unique'
isnull = td001_sys_ecs.sys_ecs_secondaire_nb_generateur.isnull()
td001_sys_ecs.loc[~isnull, 'configuration_sys_ecs'] = 'générateurs multiples/installation unique'
td001_sys_ecs.loc[(~isnull) & (
    is_multiple_install), 'configuration_sys_ecs'] = 'générateurs multiples/installation multiples'

cols_first = [el for el in td001_sys_ecs.columns.tolist() if el not in cols_end]

cols = cols_first + cols_end



# extract fiches techniques

In [ ]:
td005_ecs=td005.loc[td005.tr011_sous_categorie_fiche_technique_id=='17']
vr_ecs = td005_ecs.valeur_renseignee.str.lower().apply(lambda x:strip_accents(x))



mixte = ('mixte', 'combine', 'chauffage + ecs','chauffage et ecs','lie ','combine ')
gen_ecs_normalized_lib_matching_dict_ft = {
    "ballon thermodynamique electrique": [('thermodynamique', 'air extrait', 'air exterieur'),
                                          ('thermodynamique', 'air extrait', 'air exterieur'),

                                          ],
    "pompe a chaleur combinee ecs/chauffage (indeterminee)": [
        ('pac', 'pompe a chaleur'),
        mixte
    ],
    "pompe a chaleur combinee ecs/chauffage (pac air/eau)": [
        ('pac', 'pompe a chaleur'),
        mixte, 'air/eau'
    ],
    "pompe a chaleur combinee ecs/chauffage (pac geothermique)": [
        ('pac', 'pompe a chaleur'),
        mixte, 'geothermique'
    ],

    "pompe a chaleur combinee ecs/chauffage (pac air/air)": [
        ('pac', 'pompe a chaleur'),
        ('mixte', 'combine'), ('air/air', 'split')
    ],
    "pompe a chaleur combinee ecs/chauffage (pac eau/eau)": [
        ('pac', 'pompe a chaleur'),
        ('mixte', 'combine'), 'eau/eau'
    ],
    "ballon electrique": [('electricite', 'electrique','elec')],
    "ecs instantanee electrique": [('instantanee', 'instantane'), ('electricite', 'electrique')],

    'chaudiere mixte gaz': ["chaudiere", mixte, ("gaz", "gpl")],
    'chaudiere mixte fioul': ["chaudiere", mixte, "fioul"],
    'chaudiere mixte bois': ["chaudiere", mixte, "classe",
                             ("bois", "biomasse")],

    'chaudiere mixte indeterminee': ["chaudiere", mixte],
    'production mixte indeterminee': [ mixte],
    'chauffe-eau gaz independant': [("ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gaz"],
    'chauffe-eau gpl independant': [("ballon", "chauffe-eau", "accumulateur", "chauffe bain"), "gpl", 'gpl'],

    'chauffe-eau fioul independant': [("ballon", "chauffe-eau", "accumulateur", "bain"), "fioul", 'fioul'],
    "ecs collective reseau de chaleur": ["reseau de chaleur"],

    'chaudiere gaz': ["chaudiere", "gaz"],
    'chaudiere gpl': ["chaudiere", "gpl", 'gpl'],
    'chaudiere fioul': ["chaudiere", "fioul", 'fioul'],
    'chaudiere indeterminee': ["chaudiere"],

}

libs=vr_ecs.apply(lambda x:affect_lib_by_matching_score(x,gen_ecs_normalized_lib_matching_dict_ft))

test=libs=='pompe a chaleur combinee ecs/chauffage (indeterminee)'
test=libs=='pompe a chaleur combinee ecs/chauffage (pac air/air)'
test=libs=='ballon electrique'
test=libs=='chaudiere mixte gaz'
test=libs=='non affecte'
test=libs=='ecs collective reseau de chaleur'
test=libs=='chaudiere gaz'
test=libs=='ballon thermodynamique electrique'
test=libs=='chaudiere fioul'
test=libs=='chauffe-eau fioul independant'

#test=libs=='ecs collective reseau de chaleur'
#test=libs=='ecs collective reseau de chaleur'
#test=libs=='ecs collective reseau de chaleur'

test=libs=='chaudiere fioul'


libs.value_counts()

vr_ecs.loc[test].head(100).to_dict()